In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import metrics


In [2]:
#import the dataset from GitHub
url = 'https://raw.githubusercontent.com/jamesdinardo/Retail-Forecasting/master/final_df.csv'
df = pd.read_csv(url)

In [3]:
print('Shape of final dataframe: {}'.format(df.shape))
df.head()

Shape of final dataframe: (418660, 16)


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Type,Size,Weekly_Sales
0,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,1,A,151315,24924.50
1,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,2,A,151315,50605.27
2,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,3,A,151315,13740.12
3,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,4,A,151315,39954.04
4,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,5,A,151315,32229.38


In [4]:
#convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

#convert Store, Dept, and Type columns to category
df['Store'] = df['Store'].astype('category')

df['Dept'] = df['Dept'].astype('category')

df['Type'] = df['Type'].astype('category')

In [5]:
#create separate features for Week, Month, and Year
df['Day'] = df['Date'].dt.day
df['Day'] = df['Day'].astype('category')

df['Week'] = df['Date'].dt.week
df['Week'] = df['Week'].astype('category')

df['Month'] = df['Date'].dt.month
df['Month'] = df['Month'].astype('category')

df['Year'] = df['Date'].dt.year
df['Year'] = df['Year'].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418660 entries, 0 to 418659
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         418660 non-null  category      
 1   Date          418660 non-null  datetime64[ns]
 2   Temperature   418660 non-null  float64       
 3   Fuel_Price    418660 non-null  float64       
 4   MarkDown1     418660 non-null  float64       
 5   MarkDown2     418660 non-null  float64       
 6   MarkDown3     418660 non-null  float64       
 7   MarkDown4     418660 non-null  float64       
 8   MarkDown5     418660 non-null  float64       
 9   CPI           418660 non-null  float64       
 10  Unemployment  418660 non-null  float64       
 11  IsHoliday     418660 non-null  bool          
 12  Dept          418660 non-null  category      
 13  Type          418660 non-null  category      
 14  Size          418660 non-null  int64         
 15  Weekly_Sales  418

In [6]:
#save a copy of the full dataframe before we start changing features
df_all = pd.DataFrame(df, columns=df.columns)
df_all.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Type,Size,Weekly_Sales,Day,Week,Month,Year
0,1,2010-05-02,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,1,A,151315,24924.50,2,17,5,2010
1,1,2010-05-02,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,2,A,151315,50605.27,2,17,5,2010
2,1,2010-05-02,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,3,A,151315,13740.12,2,17,5,2010
3,1,2010-05-02,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,4,A,151315,39954.04,2,17,5,2010
4,1,2010-05-02,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,5,A,151315,32229.38,2,17,5,2010


In [7]:
#drop the date and day column
df.drop(['Date', 'Day'], axis=1, inplace=True)

In [8]:
mapping = {True:1, False:0}
df['IsHoliday'] = df['IsHoliday'].map(mapping)
df['IsHoliday'] = df['IsHoliday'].astype('category')

In [9]:
df_dummies = pd.get_dummies(df)

In [10]:
df_dummies.columns[120:]

Index(['Dept_77', 'Dept_78', 'Dept_79', 'Dept_80', 'Dept_81', 'Dept_82',
       'Dept_83', 'Dept_85', 'Dept_87', 'Dept_90', 'Dept_91', 'Dept_92',
       'Dept_93', 'Dept_94', 'Dept_95', 'Dept_96', 'Dept_97', 'Dept_98',
       'Dept_99', 'Type_A', 'Type_B', 'Type_C', 'Week_1', 'Week_2', 'Week_3',
       'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8', 'Week_9', 'Week_10',
       'Week_11', 'Week_12', 'Week_13', 'Week_14', 'Week_15', 'Week_16',
       'Week_17', 'Week_18', 'Week_19', 'Week_20', 'Week_21', 'Week_22',
       'Week_23', 'Week_24', 'Week_25', 'Week_26', 'Week_27', 'Week_28',
       'Week_29', 'Week_30', 'Week_31', 'Week_32', 'Week_33', 'Week_34',
       'Week_35', 'Week_36', 'Week_37', 'Week_38', 'Week_39', 'Week_40',
       'Week_41', 'Week_42', 'Week_43', 'Week_44', 'Week_45', 'Week_46',
       'Week_47', 'Week_48', 'Week_49', 'Week_50', 'Week_51', 'Week_52',
       'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
       'Month_7', 'Month_8', 'Month_9', '

In [11]:
X = df_dummies.drop('Weekly_Sales', axis=1).values
y = df_dummies['Weekly_Sales'].values.reshape(-1, 1)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [13]:
reg = LinearRegression()

In [14]:
reg.fit(X_train, y_train)

LinearRegression()

In [15]:
y_pred = reg.predict(X_test)

print('R2: {}'.format(metrics.r2_score(y_test, y_pred)))
print('RMSE: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

R2: 0.670992212176619
RMSE: 12947.252197022004


Next, we'll try putting our data on a log scale to see if it improves model performance

In [16]:
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p)
transformer.fit(X)

X_log = transformer.fit_transform(X)

In [17]:
X[0][:10]

array([4.23100000e+01, 2.57200000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.11096358e+02,
       8.10600000e+00, 1.51315000e+05])

In [18]:
X_log[0][:10]

array([ 3.76838356,  1.27312566,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  5.35704069,  2.20893354, 11.92712564])

In [19]:
X_train_log, X_test_log, y_train, y_test = train_test_split(X_log, y, test_size=0.25, random_state=0)

In [20]:
reg = LinearRegression()

In [21]:
reg.fit(X_train_log, y_train)

LinearRegression()

In [22]:
y_pred = reg.predict(X_test_log)

print('R2 with log scaled data: {}'.format(metrics.r2_score(y_test, y_pred)))
print('RMSE with log scaled data: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

R2 with log scaled data: 0.6710499077295844
RMSE with log scaled data: 12946.116917472


Now we'll try standard scaler instead

In [23]:
#use standard scaler on numeric columns
df[['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size']] = StandardScaler().fit_transform(df[['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size']])

In [24]:
df.head()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Type,Size,Weekly_Sales,Week,Month,Year
0,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,1,A,0.239629,24924.50,17,5,2010
1,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,2,A,0.239629,50605.27,17,5,2010
2,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,3,A,0.239629,13740.12,17,5,2010
3,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,4,A,0.239629,39954.04,17,5,2010
4,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,5,A,0.239629,32229.38,17,5,2010


In [25]:
df_dummies = pd.get_dummies(df)

In [26]:
X = df_dummies.drop('Weekly_Sales', axis=1).values
y = df_dummies['Weekly_Sales'].values.reshape(-1, 1)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [28]:
reg = LinearRegression()

In [29]:
reg.fit(X_train, y_train)

LinearRegression()

In [30]:
print('R2 on scaled test data: {}'.format(reg.score(X_test, y_test)))
print('R2 on scaled training data: {}'.format(reg.score(X_train, y_train)))

R2 on scaled test data: 0.6709924421320985
R2 on scaled training data: 0.6605720223116988


It turns out that scaling has a minimal effect on model performance. In addition, the performance on training and test data is similar, which suggests that our model is probably not overfitting. Nonetheless, we can check use L1 and L2 regularization to prevent overfitting, just in case.

In [31]:
lasso = Lasso(alpha=0.1)

In [32]:
lasso.fit(X_train, y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19716689489316.14, tolerance: 16290747583.787714
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [33]:
print('R2 using L1 regularization on scaled test data: {}'.format(lasso.score(X_test, y_test)))
print('R2 using L1 regularization on scaled training data: {}'.format(lasso.score(X_train, y_train)))

R2 using L1 regularization on scaled test data: 0.6709989797929024
R2 using L1 regularization on scaled training data: 0.6605644472650358


In [34]:
lasso.coef_.shape

(208,)

In [35]:
df_dummies.drop('Weekly_Sales', axis=1).columns

Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size',
       ...
       'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11',
       'Month_12', 'Year_2010', 'Year_2011', 'Year_2012'],
      dtype='object', length=208)

In [36]:
pd.DataFrame({'col1':lasso.coef_, 'col2':df_dummies.drop('Weekly_Sales', axis=1).columns}).sort_values(by='col1', ascending=False)

,col1,col2
130,63311.334210,Dept_92
133,58096.803884,Dept_95
93,49251.529431,Dept_38
117,38462.901153,Dept_72
114,34517.610033,Dept_65
...,...,...
105,-15977.561212,Dept_50
103,-16316.519286,Dept_48
119,-17460.519470,Dept_77
137,-17848.331029,Dept_99


Lasso reduces all coefficients to 0 except for size, which as we saw before, had the highest correlation to the target

In [37]:
ridge = Ridge(alpha=0.1)

In [38]:
ridge.fit(X_train, y_train)

Ridge(alpha=0.1)

In [39]:
print('R2 using L2 regularization on scaled test data: {}'.format(ridge.score(X_test, y_test)))
print('R2 using L2 regularization on scaled training data: {}'.format(ridge.score(X_train, y_train)))

R2 using L2 regularization on scaled test data: 0.6709926783074811
R2 using L2 regularization on scaled training data: 0.6605719832399239


In [40]:
X_train.shape

(313995, 208)

In [41]:
pd.DataFrame({'col1':ridge.coef_.reshape(208,), 'col2':df_dummies.drop('Weekly_Sales', axis=1).columns}).sort_values(by='col1', ascending=False)

,col1,col2
130,62394.331412,Dept_92
133,57179.897184,Dept_95
93,48336.141313,Dept_38
117,37546.385021,Dept_72
114,33871.066697,Dept_65
...,...,...
105,-16927.225594,Dept_50
103,-17263.976588,Dept_48
119,-18669.802476,Dept_77
137,-18827.134027,Dept_99


As with KNN, the most significant columns are department numbers

Finally, let's see if we reduce the dimensionality of the data and maintain similar model performance. We'll try the following:

1. only use months and year
2. use month, year, and week of month
3. use only the top 10 most significant departments based on largest coefficients

In [42]:
#remove week columns, so that the only date information is month and year
list_of_week_columns = list(df_dummies.columns[df_dummies.columns.str.contains('Week_')])
df_dummies_no_weeks = df_dummies.drop(list_of_week_columns, axis=1)
df_dummies_no_weeks.shape

(418660, 157)

In [43]:
X = df_dummies_no_weeks.drop('Weekly_Sales', axis=1).values
y = df_dummies_no_weeks['Weekly_Sales'].values.reshape(-1, 1)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [45]:
ridge = Ridge(alpha=0.1)

In [46]:
ridge.fit(X_train, y_train)

Ridge(alpha=0.1)

In [47]:
y_pred = ridge.predict(X_test)

print('R2 using L2 regularization on scaled test data, no weeks: {}'.format(metrics.r2_score(y_test, y_pred)))
print('RMSE using L2 regularization on scaled test data, no weeks: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

R2 using L2 regularization on scaled test data, no weeks: 0.6678331249112066
RMSE using L2 regularization on scaled test data, no weeks: 13009.262561169746


Removing the week information (52 features) results in only slightly worse model performance. We can add a new column, week_of_month, that indicates whether a week was the first, second, etc. week in a month. This will add only 5 features, instead of 52, and makes our data less redundant.

In [48]:
df_all['Day'] = df_all['Day'].astype('int')

week_of_month_list = []

for lab, row in df_all.iterrows():
    if (row['Day'] >= 1) & (row['Day'] <= 7):
        week_of_month_list.append('First Week')
    elif (row['Day'] >= 8) & (row['Day'] <= 14):
        week_of_month_list.append('Second Week')
    elif (row['Day'] >= 15) & (row['Day'] <= 21):
        week_of_month_list.append('Third Week')
    elif (row['Day'] >= 22) & (row['Day'] <= 28):
        week_of_month_list.append('Fourth Week')
    elif row['Day'] >= 29:
        week_of_month_list.append('Fifth Week')


In [49]:
df['Week of Month'] = week_of_month_list

In [50]:
df.head()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Type,Size,Weekly_Sales,Week,Month,Year,Week of Month
0,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,1,A,0.239629,24924.50,17,5,2010,First Week
1,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,2,A,0.239629,50605.27,17,5,2010,First Week
2,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,3,A,0.239629,13740.12,17,5,2010,First Week
3,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,4,A,0.239629,39954.04,17,5,2010,First Week
4,1,-0.962699,-1.717612,-0.425322,-0.173453,-0.084858,-0.276858,-0.39326,1.018806,0.077545,0,5,A,0.239629,32229.38,17,5,2010,First Week


In [51]:
df_dummies_week_of_month = pd.get_dummies(df.drop('Week', axis=1))
df_dummies_week_of_month.shape

(418660, 162)

In [52]:
df_dummies_week_of_month.columns

Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size',
       ...
       'Month_11', 'Month_12', 'Year_2010', 'Year_2011', 'Year_2012',
       'Week of Month_Fifth Week', 'Week of Month_First Week',
       'Week of Month_Fourth Week', 'Week of Month_Second Week',
       'Week of Month_Third Week'],
      dtype='object', length=162)

In [53]:
X = df_dummies_week_of_month.drop('Weekly_Sales', axis=1).values
y = df_dummies_week_of_month['Weekly_Sales'].values.reshape(-1, 1)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [55]:
ridge = Ridge(alpha=0.1)

In [56]:
ridge.fit(X_train, y_train)

Ridge(alpha=0.1)

In [57]:
y_pred = ridge.predict(X_test)

print('R2 using L2 regularization on scaled test data, week of the month: {}'.format(metrics.r2_score(y_test, y_pred)))
print('RMSE using L2 regularization on scaled test data, week of the month: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

R2 using L2 regularization on scaled test data, week of the month: 0.66808544496466
RMSE using L2 regularization on scaled test data, week of the month: 13004.320585209301


In [58]:
#get the feature names for dataframe with weeks of the month
column_names_week_of_month = df_dummies_week_of_month.drop('Weekly_Sales', axis=1).columns

In [59]:
column_names_week_of_month

Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size',
       ...
       'Month_11', 'Month_12', 'Year_2010', 'Year_2011', 'Year_2012',
       'Week of Month_Fifth Week', 'Week of Month_First Week',
       'Week of Month_Fourth Week', 'Week of Month_Second Week',
       'Week of Month_Third Week'],
      dtype='object', length=161)

In [60]:
#create a dataframe that shows the coefficient for each feature
coef_df = pd.DataFrame({'feature':column_names_week_of_month, 'coefficient':ridge.coef_[0]})
len(coef_df)

161

In [61]:
#isolate departments with significiant coefficients
coef_df.loc[(np.abs(coef_df['coefficient']) > 10000) & (coef_df['feature'].str.contains('Dept'))]

,feature,coefficient
58,Dept_2,30672.392821
60,Dept_4,13210.240526
63,Dept_7,11651.227760
64,Dept_8,17363.011552
69,Dept_13,17859.322239
74,Dept_19,-13852.674409
82,Dept_27,-12695.205001
83,Dept_28,-12761.888503
85,Dept_30,-10891.067357
86,Dept_31,-11213.741207


In [62]:
#create a list of departments to drop based on coefficients
dept_to_drop = coef_df.loc[(np.abs(coef_df['coefficient']) < 10000) & (coef_df['feature'].str.contains('Dept'))]
list_of_dept_to_drop = list(dept_to_drop['feature'])
print(f'Number of department columns to drop: {len(list_of_dept_to_drop)}')
display(list_of_dept_to_drop)

Number of department columns to drop: 39


['Dept_1',
 'Dept_3',
 'Dept_5',
 'Dept_6',
 'Dept_9',
 'Dept_10',
 'Dept_11',
 'Dept_12',
 'Dept_14',
 'Dept_16',
 'Dept_17',
 'Dept_18',
 'Dept_20',
 'Dept_21',
 'Dept_22',
 'Dept_23',
 'Dept_24',
 'Dept_25',
 'Dept_26',
 'Dept_29',
 'Dept_32',
 'Dept_33',
 'Dept_34',
 'Dept_42',
 'Dept_44',
 'Dept_46',
 'Dept_49',
 'Dept_55',
 'Dept_67',
 'Dept_71',
 'Dept_74',
 'Dept_79',
 'Dept_80',
 'Dept_81',
 'Dept_82',
 'Dept_87',
 'Dept_96',
 'Dept_97',
 'Dept_98']

In [63]:
df_dummies_week_of_month_best_dept = df_dummies_week_of_month.drop(list_of_dept_to_drop, axis=1)
df_dummies_week_of_month_best_dept.shape

(418660, 123)

In [64]:
df_dummies_week_of_month.columns

Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size',
       ...
       'Month_11', 'Month_12', 'Year_2010', 'Year_2011', 'Year_2012',
       'Week of Month_Fifth Week', 'Week of Month_First Week',
       'Week of Month_Fourth Week', 'Week of Month_Second Week',
       'Week of Month_Third Week'],
      dtype='object', length=162)

In [72]:
X = df_dummies_week_of_month_best_dept.drop('Weekly_Sales', axis=1).values
y = df_dummies_week_of_month_best_dept['Weekly_Sales'].values.reshape(-1, 1)

y = y.reshape(len(y))

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [74]:
ridge = Ridge(alpha=0.1)

In [75]:
ridge.fit(X_train, y_train)

Ridge(alpha=0.1)

In [76]:
y_pred = ridge.predict(X_test)

print('R2 using L2 regularization on scaled test data, week of the month, best dept: {}'.format(metrics.r2_score(y_test, y_pred)))
print('RMSE using L2 regularization on scaled test data, week of the month, best dept: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

R2 using L2 regularization on scaled test data, week of the month, best dept: 0.6291037700855815
RMSE using L2 regularization on scaled test data, week of the month, best dept: 13746.77194764753


In [77]:
print(X_train.shape)
print(X_test.shape)

(313995, 122)
(104665, 122)


With 121 features, we still end up with similar results to the models with all features.